In [ ]:
import pandas as pd
import plotly.express as px
import statsmodels.api as sm


In [ ]:
pd.read_json('data/Portugal/morbilidade-e-mortalidade-hospitalar.json')

In [ ]:
import json
with open('data/Portugal/morbilidade-e-mortalidade-hospitalar.json','r') as f:
    data = json.loads(f.read())

In [ ]:
data

In [ ]:
# Normalizing data
df =pd.json_normalize(data, record_path =None, meta =['datasetid','recordid','record_timestamp'], meta_prefix='config_params_', record_prefix='values_')


In [ ]:
df['fields.periodo']=pd.to_datetime(df['fields.periodo'], format='%Y-%m')

In [ ]:
df

In [ ]:
df[['fields.periodo','fields.faixa_etaria','fields.desc_capitulo','fields.obitos']].to_csv('mortalidade_hospitalar.csv')

In [ ]:
df['fields.desc_capitulo'].value_counts()

In [ ]:
dfg=df.groupby(['fields.periodo','fields.desc_capitulo'], as_index=False).agg({'fields.obitos':'sum'})

In [ ]:
dfg

In [ ]:
import plotly.express as px

fig = px.line(dfg, x=dfg['fields.periodo'], y=['fields.obitos'], color='fields.desc_capitulo')
fig.show()

In [ ]:
dates=['2014','2015','2016', '2017' , '2018' ,'2019' ,'2020', '2021']

In [ ]:
df2=pd.read_csv("data/Portugal/Dados_SICO_2021-01-12_2014.csv")

In [ ]:
def merging_dates(list_dates):
    df=pd.read_csv(f'data/Portugal/Dados_SICO_2021-01-12_{list_dates[0]}.csv').copy()
    df['Data (mm-dd)'] = df['Data (mm-dd)'].replace(regex={'Jan': '1', 'Fev': '2', 'Mar': '3', 'Abr': '4', 'Mai': '5', 'Jun': '6',
                                       'Jul': '7', 'Ago': '8', 'Set': '9', 'Out': '10', 'Nov': '11', 'Dez': '12'})
    df=df.assign(date = pd.to_datetime(df['Data (mm-dd)']+'-'+list_dates[0], format='%m-%d-%Y'))
    for date in list_dates[1:]:
        df2=pd.read_csv(f'data/Portugal/Dados_SICO_2021-01-12_{date}.csv').copy()
        df2 = df2.dropna()
        df2['Data (mm-dd)'] = df2['Data (mm-dd)'].replace(regex={'Jan': '1', 'Fev': '2', 'Mar': '3', 'Abr': '4', 'Mai': '5', 'Jun': '6',
                                       'Jul': '7', 'Ago': '8', 'Set': '9', 'Out': '10', 'Nov': '11', 'Dez': '12'})
        df2=df2.assign(date = pd.to_datetime(df2['Data (mm-dd)']+'-'+date, format='%m-%d-%Y'))
        df=pd.concat([df,df2])
        
    #df=df.sort_values(by='date').reset_index(drop=True)
    df=df.set_index('date')
    
    return df
        

In [ ]:
df2=merging_dates(dates)

In [ ]:
import plotly.express as px

fig = px.line(df2, x=df2.index, y=['< 1 ano', '1-4 anos'])
fig.show()

In [ ]:
df2=pd.melt(df,id_vars=["Data"])
df2=df2.dropna()

## 

In [ ]:
df2

In [ ]:
df2.Data=df2.Data.replace(regex={'Jan': '1', 'Fev': '2', 'Mar': '3', 'Abr': '4',  'Mai': '5',  'Jun': '6',
          'Jul': '7', 'Ago': '8', 'Set': '9', 'Out': '10', 'Nov': '11', 'Dez': '12'})


In [ ]:
df2['date']=df2["Data"]+'-'+df2["variable"].astype(str)

In [ ]:
df2.date=pd.to_datetime(df2.date, format="%m-%d-%Y")
df2=df2.set_index('date')

In [ ]:


res = sm.tsa.seasonal_decompose(df2.value,model='multiplicative',period=365,extrapolate_trend='freq')

In [ ]:
#result=res.fit()

In [ ]:
fig = res.plot()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig2 = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=df2.index, y=df2.value,
                    mode='markers',
                    name='Deaths'))
fig.add_trace(go.Scatter(x=res.trend.index, y=res.trend,
                    mode='lines',
                    name='trend'))
fig.add_trace(go.Scatter(x=res.seasonal.index, y=res.seasonal,
                    mode='lines',
                    name='Seasonality'))

fig2.add_trace(go.Scatter(x=res.resid.index, y=res.resid,
                    mode='lines+markers',
                    name='Noise'))

fig.show()
fig2.show()


In [ ]:
res.seasonal

In [ ]:
figo = px.line(res.seasonal,title="Mortalidade diária em Portugal (números oficiais)")
figo.show()

In [ ]:
fig.show()

In [ ]:
import statsmodels.api as sm

In [ ]:
res = sm.tsa.seasonal_decompose(df2.value)
fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(10,10))
res.trend.plot(ax=ax1)
res.resid.plot(ax=ax2)
res.seasonal.plot(ax=ax3)

In [ ]:
df2

In [ ]:
fig = px.line(df2, x=df2.index, y="value")

In [ ]:
fig.show()

In [ ]:
df=pd.read_csv("data/Deutschland/sterbefallzahlen.csv", delimiter=';',usecols=range(6))#, skiprows=7, nrows=60, usecols=range(0,5))

In [ ]:
df

In [ ]:
df2=pd.melt(df,id_vars=["Kalenderwoche"])
df2

In [ ]:
df2['date']=df2.Kalenderwoche.astype(str)+'-'+df2.variable.astype(str)
df2['date']

In [ ]:
df2['date'] = pd.to_datetime(df2.Kalenderwoche.astype(str)+ df2.variable.astype(str).add('-1') ,format='%V%G-%u')

In [ ]:
df2['date'][100]

In [ ]:
fig = px.line(df2, x=df2.date, y="value")
fig.show()

In [ ]:
df=pd.read_csv("data/Deutschland/Bevölkerung.csv", delimiter=';',skiprows=5,nrows=70, encoding='ISO-8859-1')
df

In [ ]:
df['Unnamed: 0']=pd.to_datetime(df['Unnamed: 0'])

In [ ]:

fig = px.line(df, x='Unnamed: 0', y="Anzahl")
fig.show()

In [ ]:
df=pd.read_csv("data/Deutschland/sterbefallzahlen_monatlich.csv", delimiter=';',skiprows=6,nrows=369, encoding='ISO-8859-1')
df

In [ ]:
md={'Januar':1,'Februar':2,'März':3,'April':4,'Mai':5,'Juni':6,'Juli':7,'August':8,'September':9,'Oktober':10,'November':11,'Dezember':12}
#month
df["Unnamed: 1"] = df["Unnamed: 1"].map(md).astype(int)
df

In [ ]:
df['date']=df["Unnamed: 1"].astype(str)+'-'+df["Unnamed: 0"].astype(str)

In [ ]:
df.date[11]

In [ ]:
df.date=pd.to_datetime(df.date, format='%m-%Y')
df.date

In [ ]:

fig = px.line(df, x=df.date, y="Anzahl")
fig.show()

In [ ]:
df

In [ ]:
X = sm.add_constant(df)
X

In [ ]:
y= df.Anzahl

In [ ]:
olsmod = sm.OLS(y, X['date'])
olsres = olsmod.fit()
print(olsres.summary())

In [ ]:
import numpy as np
nsample = 50
sig = 0.25
x1 = np.linspace(0, 20, nsample)
X = np.column_stack((x1, np.sin(x1), (x1-5)**2))
X

In [ ]:
df=pd.read_csv("data/Deutschland/sterbefallzahlen_Altersjahre.csv", skiprows=329,encoding='utf-8', index_col=1)
df
#df=df.loc[df['Unnamed: 0']=='Insgesamt']
#df.astype(int, errors='ignore')

In [ ]:
df.T


In [ ]:
fig3 = px.line(df.T, x=df.T.index, y=['Insgesamt','unter 1 Jahr', '1-9-Jährige', '10-19-Jährige', '20-29-Jährige',
       '30-39-Jährige', '40-49-Jährige', '50-59-Jährige', '60-69-Jährige',
       '70-79-Jährige', '80-89-Jährige', '90-99-Jährige',
       '100 Jahre und mehr'])
fig3.show()

In [ ]:
df=pd.read_csv("data/Portugal/pordata_pop_ge.csv",skiprows=7, nrows=49,usecols=range(20),index_col=0)
df.columns

In [ ]:
import plotly.express as px

fig = px.line(df, x=df.index, y=['0-04', '05-09', '10-14', '15-19', '20-24', '25-29', '30-34',
       '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74',
       '75-79', '80-84', '85 ou mais'])
fig.show()

In [ ]:
dfp=pd.read_csv("data/Deutschland/Bevölkerung_AG.csv",skiprows=102,nrows=20, encoding='utf-8',index_col=0)
dfp=dfp.T
dfp.index=pd.to_datetime(dfp.index, format='%d.%m.%Y')
dfp.columns

In [ ]:
import plotly.express as px

fig = px.line(df, x=df.index, y=['0-4-Jährige', '5-9-Jährige', '10-14-Jährige', '15-19-Jährige',
       '20-24-Jährige', '25-29-Jährige', '30-34-Jährige', '35-39-Jährige',
       '40-44-Jährige', '45-49-Jährige', '50-54-Jährige', '55-59-Jährige',
       '60-64-Jährige', '65-69-Jährige', '70-74-Jährige', '75-79-Jährige',
       '80-84-Jährige', '85 Jahre und mehr', 'Insgesamt'])
fig.show()